In [1]:
import json
import requests
import datetime
import time
import pytz
import pandas as pd
import pprint as pp
import matplotlib.pyplot as plt

In [10]:
from badassdatascience.forex.utilities.oanda_tools import get_oanda_headers
from badassdatascience.forex.utilities.oanda_tools import price_type_map

In [3]:
# This cell is tagged `parameters`

config_file = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forex/data/DEVELOPMENT.json'
count = 5000 
granularity = 'D'
now = True

#instruments = 'EUR_USD,USD_CAD,USD_JPY,USD_CHF,AUD_USD,GBP_USD,NZD_USD'
instrument = 'EUR_USD'

price_types = 'BAM'
error_retry_interval = 5

In [4]:
print(count)
print(granularity)

5000
D


In [6]:
if True:
    def deal_with_candlestick_format_and_time(candle, timezone, price_type_list):
        candle['time'] = int(float(candle['time']))
        time_dt = datetime.datetime.fromtimestamp(candle['time'], tz = timezone)
        candle['time_iso'] = time_dt.isoformat()
        candle['weekday'] = time_dt.weekday()
        candle['hour'] = time_dt.hour

        #
        # deal with prices that are currently string values but need to be float
        #
        # and reorganize them
        #
        for price_type in price_type_list:
            for candlestick_component in candle[price_type].keys():
                candle[price_type + '_' + candlestick_component] = float(candle[price_type][candlestick_component])
            candle[price_type + '_return'] = candle[price_type + '_c'] - candle[price_type + '_o']
            candle[price_type + '_volatility'] = candle[price_type + '_h'] - candle[price_type + '_l']
            
        for price_type in price_type_list:
            del(candle[price_type])
            
        #return None

In [7]:
# no previous

class test():
    def __init__(
        self,
        config_file,
        count,
        granularity,
        now,
        instrument,
        price_types,
        error_retry_interval = 5,
        keep_complete_only = True
    ):

        # command line arguments
        self.config_file = config_file
        self.count = count
        self.granularity = granularity
        self.now = now
        self.instrument = instrument
        self.price_types = price_types
        self.error_retry_interval = error_retry_interval
        self.keep_complete_only = keep_complete_only

        # initialize
        self.timezone_to_use = 'America/Toronto'   # Don't change this!
        self.start_time = int(datetime.datetime(2010, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc).timestamp())
        
        
        self.end_date = None
        self.output_file_sans_extension = 'output/' + granularity
        
        self.timezone = pytz.timezone(self.timezone_to_use)  # not sure we are using this... check later
        self.price_type_list = [price_type_map[q] for q in self.price_types]
                
        self.end_date_original = int(time.mktime(datetime.datetime.now().timetuple()))
        

    def load_config(self):
        with open(self.config_file) as f:
            self.config = json.load(f)

    def get_headers(self):
        self.headers = get_oanda_headers(self.config)


    def get_instrument_candlesticks(self, end_date):
        url = (
            self.config['server']
            + '/v3/instruments/' + self.instrument
            + '/candles?count=' + str(self.count)
            + '&price=' + self.price_types
            + '&granularity=' + self.granularity
            + '&to=' + str(end_date)
        )
        
        worked = False
        while not worked:
            try:
                r = requests.get(url, headers = self.headers)
                worked = True
            except:
                time.sleep(error_retry_interval)
        
        rj = r.json()
        return rj





    def compute_candle_features(self):
        
        finished = False
        end_date = self.end_date_original

        self.insert_many_list = []

        # loop through the timestamp ranges for each set of n=count values
        while not finished:

            # retrieve the instrument candlesticks from the Oanda server
            rj = self.get_instrument_candlesticks(end_date) # instrument, count, price_types, granularity, end_date)        
            candlesticks = rj['candles']
            
            # deal with timestamps and time-related content
            date_list = []
            for candle in candlesticks:
                deal_with_candlestick_format_and_time(candle, self.timezone, self.price_type_list)
                date_list.append(candle['time'])
                
            rj['timestamp_int_min'] = min(date_list)
            rj['timestamp_int_max'] = max(date_list)


            for candle in rj['candles']:
                candle['instrument'] = self.instrument
                candle['granularity'] = self.granularity

                if self.keep_complete_only:
                    if candle['complete']:    
                        self.insert_many_list.append(candle)
                else:
                    self.insert_many_list.append(candle)
            

            # Are we done?
            if (len(date_list) < count) or (min(date_list) < self.start_time):
                finished = True
            
            # prepare for the next iteration
            end_date = rj['timestamp_int_min'] - 0.1


    def create_dataframe(self):
        self.df = pd.DataFrame(self.insert_many_list).sort_values(by = ['instrument', 'time'])
        self.df = self.df[self.df['time'] >= int(self.start_time)]
        self.df = self.df.reset_index().copy()

    def fit(self):
        self.load_config()
        self.get_headers()
        self.compute_candle_features()
        self.create_dataframe()
        

In [8]:
t = test(config_file, count, granularity, now, instrument, price_types, error_retry_interval = 5)
t.fit()

In [9]:
t.df

,index,complete,volume,time,time_iso,weekday,hour,bid_o,bid_h,bid_l,...,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,instrument,granularity
0,738,True,1027,1262469600,2010-01-02T17:00:00-05:00,5,17,1.43070,1.43325,1.42909,...,-0.00114,0.00581,1.43120,1.43425,1.42926,1.43036,-0.00084,0.00499,EUR_USD,D
1,739,True,42031,1262556000,2010-01-03T17:00:00-05:00,6,17,1.43010,1.44556,1.42569,...,0.01086,0.01982,1.43030,1.44560,1.42576,1.44127,0.01097,0.01984,EUR_USD,D
2,740,True,45159,1262642400,2010-01-04T17:00:00-05:00,0,17,1.44114,1.44833,1.43464,...,-0.00472,0.01372,1.44123,1.44839,1.43468,1.43650,-0.00473,0.01371,EUR_USD,D
3,741,True,45142,1262728800,2010-01-05T17:00:00-05:00,1,17,1.43655,1.44341,1.42824,...,0.00412,0.01510,1.43665,1.44346,1.42832,1.44078,0.00413,0.01514,EUR_USD,D
4,742,True,42005,1262815200,2010-01-06T17:00:00-05:00,2,17,1.44065,1.44451,1.42992,...,-0.00994,0.01475,1.44074,1.44464,1.42996,1.43079,-0.00995,0.01468,EUR_USD,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4256,4994,True,238236,1744664400,2025-04-14T17:00:00-04:00,0,17,1.13429,1.13784,1.12633,...,-0.00700,0.01152,1.13479,1.13792,1.12640,1.12819,-0.00660,0.01152,EUR_USD,D
4257,4995,True,289974,1744750800,2025-04-15T17:00:00-04:00,1,17,1.12802,1.14121,1.12787,...,0.01132,0.01313,1.12832,1.14130,1.12810,1.13986,0.01154,0.01320,EUR_USD,D
4258,4996,True,255008,1744837200,2025-04-16T17:00:00-04:00,2,17,1.13979,1.14086,1.13344,...,-0.00347,0.00742,1.13994,1.14094,1.13352,1.13646,-0.00348,0.00742,EUR_USD,D
4259,4997,True,198710,1744923600,2025-04-17T17:00:00-04:00,3,17,1.13623,1.13969,1.13567,...,0.00274,0.00395,1.13673,1.13978,1.13592,1.13950,0.00277,0.00386,EUR_USD,D
